baizer::pkglib(tidyverse, Biostrings, baizer, genogamesh, yaml)

sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

Pcount <- snakemake@input[['VDJB_count_dir']]
Pigblast_tsv <- snakemake@input[['VDJB_igblast_tsv']]
Pigblast_txt <- snakemake@input[['VDJB_igblast_txt']]
Pchangeo <- snakemake@input[['VDJB_changeo']]

Pcsv <- snakemake@output[['VDJB_csv']]
Pstat <- snakemake@output[['VDJB_stat']]

# read from cellranger
TBcr_airr <- read_tsv(str_glue('{Pcount}/airr_rearrangement.tsv'))
TBcr_anno <- read_csv(str_glue('{Pcount}/all_contig_annotations.csv'))

# read from igblast
TBigblast_tsv <- read_tsv(Pigblast_tsv)
TBshm <- parse_IgBlast_shm(Pigblast_txt)

# read from changeo
TBchangeo <- read_tsv(Pchangeo)